# Maybe we're getting the wrong modes in the increasing T N2 -> N1 study

Here we pick a good wavelength and check the mode profiles to ensure we're not incorrect.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from scipy.optimize import newton
from ipywidgets import interactive, FloatSlider, Layout

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
outer = 'h2'
nu = 1

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]
maxhs = [1,1,1,1]
d = 4.0775e-05
T = 1
ts = [d , 1e-5, T * d, 1.5e-5]

H = BraggExact(ts=ts, ns=ns, maxhs=maxhs)

# Wavelength to check is 1.8e-6

From graphs in Wavelength_N2_increasing notebook things are solid at this wavelength for our choices of T

## Checking N2 modes

In [ ]:
nu = 1

def det_plot(T, wl):
    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]
    H.ts = ts
    H.wavelength = wl * 1e-6
    cutoff = H.ks[0] * H.scale
    
    Xr = np.linspace(.9998* cutoff , 1.000001*cutoff , num=75)
    Xi = np.linspace( -.001, .001, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, 1, figsize=(14,10))
    
    fs1 = H.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    m, M = ax1.get_ylim()
    ax1.plot([cutoff, cutoff], [m,M],lw=4, c="r")
    ax1.set_facecolor('grey')
    ax1.set_title('Behavior near Fundamental Mode as outer thickness changes.')
#     ax1.set_xticklabels([])
    ax1.set_yticks([0])
    plt.show()
    
interactive_plot = interactive(det_plot,
                               T=FloatSlider(min=1, max=3, step=.05, value=1.5, 
                                             readout_format='.2f', layout=Layout(width='90%')),
                               wl=FloatSlider(min=1.4, max=2, step=.0005, value=1.8, 
                                              readout_format='.2f', layout=Layout(width='90%')))

output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

In [ ]:

guess = np.array(52.367)

beta1 = newton(H.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(H.determinant(beta1, nu, outer)))


In [ ]:
Fs = H.all_fields(beta1, nu, outer)

In [ ]:
Draw(Fs['Etv'].real, H.mesh)

# Okay, it's all looking good

The values we calculated on the spectral loss responses are the ones we see here for the 'inner fundamental'.

## N1 modes

Think they're good but never hurts to check.  There's no thickness to vary.

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]
maxhs = [1,1,1]
mats = ['air', 'glass', 'air']
c
d = 4.0775e-05
T = 1
ts = [d , 1e-5, T * d]

H = BraggExact(ts=ts, ns=ns, maxhs=maxhs, mats=mats)

In [ ]:
nu = 1

def det_plot(T, wl):
    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d]
    H.ts = ts
    H.wavelength = wl * 1e-6
    cutoff = H.ks[0] * H.scale
    
    Xr = np.linspace(.999 * cutoff , 1.0001*cutoff , num=75)
    Xi = np.linspace( -.01, .01, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, 1, figsize=(14,10))
    
    fs1 = H.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    m, M = ax1.get_ylim()
    ax1.plot([cutoff, cutoff], [m,M],lw=4, c="r")
    ax1.set_facecolor('grey')
    ax1.set_title('Behavior near Fundamental Mode as outer thickness changes.')
#     ax1.set_xticklabels([])
    ax1.set_yticks([0])
    plt.show()
    
interactive_plot = interactive(det_plot,
                               T=FloatSlider(min=1, max=3, step=.05, value=1, 
                                             readout_format='.4f', layout=Layout(width='90%')),
                               wl=FloatSlider(min=1.4, max=2, step=.0005, value=1.8, 
                                              readout_format='.4f', layout=Layout(width='90%')))

output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

In [ ]:
52.34030162612004-4.091625724906829e-05j
guess = np.array(52.367)

beta1 = newton(H.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(H.determinant(beta1, nu, outer)))


In [ ]:
Fs = H.all_fields(beta1, nu, outer)

In [ ]:
Draw(Fs['Etv'].real, H.mesh)

# That doesn't seem like what we had before.

I thought I was getting the fundamentals easily on N1, but looks like maybe I wasn't.  This could be good.

## Has a hard time getting it

Tried with initial guesses closer to k_low, but that only works for shorter wavelengths.  Let's see if the real section technique helps

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]
maxhs = [1,1,1]
mats = ['air', 'glass', 'air']
d = 4.0775e-05
T = 1
ts = [d , 1e-5, T * d]

H = BraggExact(ts=ts, ns=ns, maxhs=maxhs, mats=mats)

In [ ]:
nu = 1

def det_plot_real(T, wl):
    d = 4.0775e-05                 # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d]  # d is base thickness, scaled by input T
    H.wavelength = wl * 1e-6
    H.ts = ts                      # set ts in BraggExact class (it updates necessary derived values)
    cutoff = H.ks[0] * H.scale     # mode cutoff, used for limits
    
    xr = np.linspace(.999* cutoff , 1.0001*cutoff , num=5000)
    h = xr[1] - xr[0]                         # h for estimating derivatives
    fs = H.determinant(xr, nu=nu, outer='h2') # function values
    fsa = np.abs(fs)                          # abs of values
    fsp = (fsa[2:] - fsa[:-2])/(2*h)          # center difference estimate for derivative of abs(fs)
    
    fspp = (fsa[2:] - 2 * fsa[1:-1] + fsa[:-2])/(h**2)  # fourth order accurate second derivative difference estimate

    dp = fsp[1:] * fsp[:-1]         # find where sign of derivative changes
    indices = np.where(dp <= 0)[0]  # great idea from stack overflow: https://stackoverflow.com/questions/45174182/finding-all-roots-in-a-given-interval-in-1d


    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,12))
    ax1.plot(xr, np.log(fsa))
    ax2.plot(xr[1:-1],fsp, [xr[1], xr[-2]], [0,0])   # derivatives exist at interior points xr[1:-1]
#     ax3.plot(xr[1:-1],fspp, [xr[1], xr[-2]], [0,0])

    for idx in indices:  # graph vertical lines at zeros of derivative
        if fspp[idx] < 0:  # filter out local maxes using 2nd derivative
            pass
        else:
            m, M = ax1.get_ylim()
            ax1.plot([xr[idx], xr[idx]], [m,M])
            m, M = ax2.get_ylim()
            ax2.plot([xr[idx], xr[idx]], [m,M])
    m, M = ax1.get_ylim()
    ax1.plot([cutoff, cutoff], [m,M],lw=4, c="r")
    ax1.grid('both', axis='both')
    ax2.grid('both', axis='both')
#     ax3.grid('both', axis='both')
    ax1.set_title('Finding propagation constants as replacement occurs.\n')
    plt.show()
    
interactive_plot = interactive(det_plot_real, T=FloatSlider(min=1, max=3, step=.05, value=1.5, 
                                             readout_format='.4f', layout=Layout(width='90%')),
                               wl=FloatSlider(min=1.4, max=2, step=.0005, value=1.8, 
                                              readout_format='.4f', layout=Layout(width='90%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot